Testing various partial derivative related membership algorithms on a randomly generated regular expression.
This is the next generation of testing beyond `docs/pdOptimizations.py` and `docs/pd_results_saved.rtf`.

In [5]:
import time
import cProfile
import pstats
from benchmark.convert import Converter

arbitrary_re = "(((((((((e g) a) ((f a) + c)) (@any (b + d?))) ((((a f) j) + @any) + ((c + ((((j? ((((h + ((d d) (([^df-g]* + d) (d [ab-cdefg-hij])))) (j? + i)) + [acd-eghj]) + (h + f))) + ((h (a + g)) + ((i + (c (c + ((c + (@any + ((((@any + @any)? c)* + h) + ((g + i) + h)))) @any))?)?) (((d + ((i j?) g)*) ((@any + ((g [c-gij])* j)) + [bde-gh-ij])) j)))) (h b)) ((a b?) + ((i + (i d)?) c*)))) d)))* d?) d) + (c? + [b-cd-ehj])) + d)"
re = Converter().math(arbitrary_re)
words = re.pairGen()
print "Generated", len(words), "accepting words with average length", sum(map(lambda w: len(w), words))*1.0/len(words)

def testit(evalFtn):
    profiler = cProfile.Profile()
    profiler.enable()
    
    t0 = time.time()
    for word in words:
        assert evalFtn(re, word)
    tf = time.time()
    
    profiler.disable()
    print "Took", tf - t0, "seconds"

    stats = pstats.Stats(profiler)
    stats.strip_dirs()
    stats.sort_stats("cumulative")
    stats.print_stats()

Generated 408 accepting words with average length 425.303921569


In [6]:
# PDDAG Construction, then evaluate word membership
testit(lambda re, word: re.nfaPDDAG().evalWordP(word))

Took 10.1835949421 seconds
         18701644 function calls (17929670 primitive calls) in 10.183 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      408    0.006    0.000   10.183    0.025 <ipython-input-6-b85210a69a57>:2(<lambda>)
      408    0.072    0.000    5.596    0.014 pddag.py:266(pddag)
      408    0.067    0.000    4.581    0.011 fa.py:2156(evalWordP)
   173524    1.133    0.000    4.512    0.000 fa_ext.py:57(evalSymbol)
      408    0.001    0.000    4.500    0.011 pddag.py:77(__init__)
61200/408    0.077    0.000    4.496    0.011 pddag.py:89(getIdx)
275808/16728    0.623    0.000    3.761    0.000 pddag.py:160(getConcatIdx)
106080/18360    0.319    0.000    3.609    0.000 pddag.py:214(catLF)
239088/35904    0.155    0.000    3.115    0.000 pddag.py:219(<setcomp>)
   735216    0.260    0.000    3.005    0.000 reex.py:378(__hash__)
   735216    0.230    0.000    2.685    0.000 {repr}
   622200    0.398    0

In [7]:
# PDRPN Construction, then evaluate word membership
testit(lambda re, word: re.nfaPDRPN().evalWordP(word))

Took 6.10067200661 seconds
         12611836 function calls (12282134 primitive calls) in 6.100 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      408    0.006    0.000    6.100    0.015 <ipython-input-7-c2f9f7ca0e1b>:2(<lambda>)
      408    0.066    0.000    4.562    0.011 fa.py:2156(evalWordP)
   173524    1.115    0.000    4.494    0.000 fa_ext.py:57(evalSymbol)
   598818    0.370    0.000    1.571    0.000 reex_ext.py:915(derivative)
      408    0.166    0.000    1.532    0.004 reex_ext.py:61(nfaPDRPN)
   791191    0.397    0.000    1.040    0.000 reex.py:1383(__init__)
   523595    0.271    0.000    0.963    0.000 reex_ext.py:879(__init__)
    52224    0.043    0.000    0.693    0.000 fa_ext.py:72(addTransition)
    52224    0.082    0.000    0.651    0.000 fa.py:1976(addTransition)
   791599    0.416    0.000    0.644    0.000 reex_ext.py:11(__init__)
    80599    0.059    0.000    0.633    0.000 reex_ext.py:11

In [8]:
# PDO Construction, then evaluate word membership
testit(lambda re, word: re.nfaPDO().evalWordP(word))

Took 336.902625084 seconds
         665095228 function calls (401687942 primitive calls) in 336.855 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      408    0.043    0.000  336.855    0.826 <ipython-input-8-38d993d52793>:2(<lambda>)
      408    0.003    0.000  332.357    0.815 reex_ext.py:56(nfaPDO)
      408    0.280    0.001  332.352    0.815 reex.py:521(nfaPDO)
175514664/2035920   49.194    0.000  326.755    0.000 {repr}
76699104/1059984   51.931    0.000  323.241    0.000 reex.py:1863(__repr__)
7720992/3084072    4.780    0.000  249.916    0.000 reex_ext.py:641(__repr__)
7720992/3084072    3.165    0.000  247.968    0.000 reex.py:2530(__repr__)
  1216345    0.665    0.000  236.745    0.000 reex.py:367(__eq__)
    16320    0.178    0.000  192.059    0.012 fa.py:255(addState)
 64455840   35.159    0.000  187.725    0.000 reex_ext.py:912(__repr__)
 70889184   99.789    0.000  151.386    0.000 reex_ext.py:900(__str__

In [9]:
# Implemented direct pd algorithm (same as re.evalWordP_PD)
def implemented(self, word):
    memo = dict() # re.rpn(): {str.sigma: dict(pd.rpn(), pd)}
    current = dict([(self.rpn(), self)])
    for sigma in word:
        nxt = dict()
        for pdstr, pd in current.items():
            if not memo.has_key(pdstr):
                memo[pdstr] = dict([(sigma, dict([(x.rpn(), x) for x in pd.partialDerivatives(sigma)]))])
            elif not memo[pdstr].has_key(sigma):
                memo[pdstr][sigma] = dict([(x.rpn(), x) for x in pd.partialDerivatives(sigma)])
            nxt.update(memo[pdstr][sigma])
        current = nxt
    for pd in current.values():
        if pd.ewp():
            return True
    return False

testit(implemented)

Took 68.6461658478 seconds
         125029112 function calls (79251249 primitive calls) in 68.571 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      408    0.523    0.001   68.571    0.168 <ipython-input-9-b4f6aeb77262>:2(implemented)
344463/41711    0.918    0.000   61.280    0.001 reex_ext.py:340(partialDerivatives)
   300896    0.138    0.000   57.463    0.000 {method 'add' of 'set' objects}
   298718    0.173    0.000   57.316    0.000 reex.py:378(__hash__)
31219573/3834627    8.562    0.000   57.216    0.000 {repr}
12049201/276519    9.706    0.000   56.599    0.000 reex.py:1863(__repr__)
1360252/734407    1.010    0.000   40.579    0.000 reex_ext.py:641(__repr__)
1360252/734407    0.642    0.000   40.026    0.000 reex.py:2530(__repr__)
  9989784    6.475    0.000   31.896    0.000 reex_ext.py:912(__repr__)
 11009475   17.190    0.000   25.339    0.000 reex_ext.py:900(__str__)
1926292/1630986    1.370    0.000   1

In [10]:
# Naive direct pd algorithm
def naive(self, word):
    current = set([self])
    for sigma in word:
        next = set()
        for pd in current:
            next.update(pd.partialDerivatives(sigma))
        current = next
    for pd in current:
        if pd.ewp():
            return True
    return False

testit(naive)

Took 438.418345928 seconds
         830354677 function calls (505872854 primitive calls) in 438.418 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      408    1.040    0.003  438.418    1.075 <ipython-input-10-c491571a2f3b>:2(naive)
2675125/317171    6.743    0.000  436.299    0.001 reex_ext.py:340(partialDerivatives)
  2356354    1.083    0.000  410.256    0.000 {method 'add' of 'set' objects}
  2351714    1.275    0.000  409.274    0.000 reex.py:378(__hash__)
212908200/2465160   62.599    0.000  406.513    0.000 {repr}
92433523/2192639   65.859    0.000  404.823    0.000 reex.py:1863(__repr__)
10664090/5876645    6.932    0.000  293.467    0.000 reex_ext.py:641(__repr__)
10664090/5876645    4.378    0.000  289.628    0.000 reex.py:2530(__repr__)
 76309086   43.384    0.000  228.784    0.000 reex_ext.py:912(__repr__)
 84030633  121.280    0.000  183.835    0.000 reex_ext.py:900(__str__)
14911904/12707190    9.406    0.

In [11]:
# direct PD algorithm, my best attempt
testit(lambda re, word: re.evalWordP_PD_Optimized(word))

Took 4.69163608551 seconds
         7776523 function calls (6467406 primitive calls) in 4.691 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      408    0.095    0.000    4.691    0.011 <ipython-input-11-dc048676e88e>:2(<lambda>)
      408    0.430    0.001    4.596    0.011 reex_ext.py:149(evalWordP_PD_Optimized)
344463/41711    1.114    0.000    3.817    0.000 reex_ext.py:353(partialDerivativesRPN)
63015/23655    0.075    0.000    1.170    0.000 reex_ext.py:489(partialDerivativesRPN)
   151999    0.190    0.000    1.027    0.000 reex_ext.py:923(partialDerivativesRPN)
   258771    0.272    0.000    0.755    0.000 reex_ext.py:308(__init__)
992856/255141    0.388    0.000    0.621    0.000 reex.py:2802(ewp)
    30181    0.050    0.000    0.511    0.000 reex_ext.py:608(partialDerivativesRPN)
   259179    0.234    0.000    0.485    0.000 reex.py:1857(__init__)
   463426    0.291    0.000    0.451    0.000 reex_ext.py:11(__